In [1]:
import numpy as np
import pandas as pd


class Matrix_Builder:

    midi_number = {'C4':60, 'Dm4':61, 'D4':62, 'Em4':63, 'E4':64, 'F4':65,
         'Gm4':66, 'G4':67, 'Am4':68, 'A4':69, 'Bm4':70, 'B4':71}
    
    def __init__(self, *songs):
        self.midi_songs = []
        for song in songs:
            self.midi_songs = self.midi_songs + [self.convert_to_midi(song)]
        
        self.pitch_relations = {}
        self.duration_relations = {}
        
        self.pitch_matrix = pd.DataFrame()
        self.duration_matrix= pd.DataFrame()
        
    def convert_to_midi(self, score): # score : n x 2 매트릭스
        
        for note in score:
            if note[0] == 'r':
                continue
                
            elif note[0][-1] == '4':
                note[0] = Matrix_Builder.midi_number[note[0]]
                
            else:
                mid_pitch = Matrix_Builder.midi_number[note[0][:-1]+ '4']
                octaved = mid_pitch + (int(note[0][-1]) - 4) * 12
                note[0] = octaved
        return score
        
    def get_relation(self):
        
        for song in self.midi_songs:
            
            for i in range(len(song)-1):
                p_relation = (song[i][0] + '-' + song[i+1][0])
                d_relation = (song[i][1] + '-' + song[i+1][1])

                if p_relation in self.pitch_relations:
                    self.pitch_relations[p_relation] = self.pitch_relations[p_relation] + 1
                else:
                    self.pitch_relations[p_relation] = 1

                if d_relation in self.duration_relations:
                    self.duration_relations[d_relation] = self.duration_relations[d_relation] + 1
                else:
                    self.duration_relations[d_relation] = 1
                
               
    def make_matrix(self):
        p_state = set()
        d_state = set()
        
        for relation in list(self.pitch_relations.keys()):
            p_state = p_state.union( set(relation.split('-')) )
        for relation in list(self.duration_relations.keys()):
            d_state = d_state.union( set(relation.split('-')) )

        self.pitch_matrix = pd.DataFrame(columns=list(p_state), index=list(p_state))
        self.duration_matrix = pd.DataFrame(columns=list(d_state), index=list(d_state))
        
        self.pitch_matrix = self.pitch_matrix.fillna(0)
        self.duration_matrix = self.duration_matrix.fillna(0)

        for state_count in self.pitch_relations.items():
            relation = state_count[0].split('-')
            prevnode = str(relation[0])
            nextnode = str(relation[1])
            self.pitch_matrix[nextnode][prevnode] = state_count[1]
        
        for state_count in self.duration_relations.items():
            relation = state_count[0].split('-')
            prevnode = str(relation[0])
            nextnode = str(relation[1])
            self.duration_matrix[nextnode][prevnode] = state_count[1]
    
        
    def show_matrix(self):
        display(self.pitch_matrix)
        display(self.duration_matrix)